# PETL-AST: Conformer Adapter on ESC-50

**Paper**: Parameter-Efficient Transfer Learning of Audio Spectrogram Transformers (Cappellazzo et al., 2024)

**Runtime**: Go to **Runtime → Change runtime type → T4 GPU** before running.

---

## 1. Setup

In [20]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch: 2.10.0+cu128
CUDA available: True
GPU: Tesla T4


In [21]:
%cd /content
!rm -rf PETL-AST-Project

# Download repo as tarball (more reliable than git clone on Colab)
!wget -q https://github.com/yuvalanavi/PETL-AST-Project/archive/refs/heads/main.tar.gz -O repo.tar.gz
!tar -xzf repo.tar.gz && mv PETL-AST-Project-main PETL-AST-Project && rm repo.tar.gz

%cd PETL-AST-Project
!ls

/content
/content/PETL-AST-Project
dataset		  evaluation.py  PETL_AST_Colab.ipynb  src
docs		  hparams	 readme.txt	       train.py
download_data.sh  main.py	 requirements.txt      utils


In [22]:
# Install dependencies (torch is already on Colab — only extra packages get installed)
# NOTE: pip may show red warnings about numpy conflicts with jax/opencv — these are harmless.
!pip install -r requirements.txt -q
!python -c "from transformers import ASTModel; print('transformers OK')"
!python -c "import librosa; print('librosa OK')"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 109.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 136.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.13.0.92 requires numpy

## 2. Download ESC-50 Dataset

In [12]:
%cd /content/PETL-AST-Project
!bash download_data.sh

bash: download_data.sh: No such file or directory


In [13]:
# Verify dataset structure
import os
audio_dir = 'data/ESC-50/audio'
meta_file = 'data/ESC-50/meta/esc50.csv'
n_files = len(os.listdir(audio_dir)) if os.path.isdir(audio_dir) else 0
print(f"Audio files: {n_files}")
print(f"Metadata exists: {os.path.isfile(meta_file)}")
assert n_files == 2000, f"Expected 2000 audio files, got {n_files}"
print("Dataset OK ✓")

Audio files: 0
Metadata exists: False


AssertionError: Expected 2000 audio files, got 0

## 3. Smoke Test (1 fold, 3 epochs)

In [ ]:
!python train.py \
    --data_path 'data' \
    --dataset_name 'ESC-50' \
    --method 'adapter' \
    --adapter_block 'conformer' \
    --adapter_type 'Pfeiffer' \
    --seq_or_par 'parallel' \
    --reduction_rate_adapter 96 \
    --kernel_size 8 \
    --device cuda \
    --num_folds 1 --num_epochs 3 \
    --save_best_ckpt True --output_path './outputs'

**Check**: Loss should decrease over 3 epochs. Trainable params should be ~271K.  
If this works, proceed to full training below.

## 4. Full Training (5 folds × 50 epochs)

**Estimated time**: ~2-3 hours on T4.

In [ ]:
# Clean smoke test outputs before full run
!rm -rf outputs/*

In [ ]:
!python train.py \
    --data_path 'data' \
    --dataset_name 'ESC-50' \
    --method 'adapter' \
    --adapter_block 'conformer' \
    --adapter_type 'Pfeiffer' \
    --seq_or_par 'parallel' \
    --reduction_rate_adapter 96 \
    --kernel_size 8 \
    --device cuda \
    --save_best_ckpt True --output_path './outputs'

## 5. Generate Convergence Plots

In [ ]:
!python utils/visualization.py --log_dir outputs --output_dir outputs/plots

In [ ]:
# Display plots inline
from IPython.display import Image, display
import glob

for img_path in sorted(glob.glob('outputs/plots/*.png')):
    print(img_path)
    display(Image(filename=img_path))

## 6. Evaluate Saved Checkpoints

In [ ]:
!python evaluation.py --data_path data --checkpoint_dir outputs --device cuda

## 7. Download Results

Download checkpoints, logs, and plots to your local machine.

In [ ]:
# Zip all outputs for download
!zip -r outputs_bundle.zip outputs/

from google.colab import files
files.download('outputs_bundle.zip')